In [1]:
import gym
from room.envs import GymEnvWrapper
from typing import List, Optional, Union, Dict
from room.common.typing import CfgType
import torch
from tqdm import trange

from room import notice
from room.agents import Agent, DQN
from room.common.callbacks import Callback
from room.common.utils import get_param
from room.envs.wrappers import EnvWrapper
from room.loggers import Logger
from room.memories import Memory, RandomMemory
from room.trainers.base import Trainer
from room.agents.policies import Policy
from room.networks.blocks import MLP
import time

In [2]:
env = gym.make("CartPole-v1")
env = GymEnvWrapper(env)

In [3]:
# class DQN(Agent):
    
#     def __init__(
#         self,
#         policy: Union[Policy, str],
#         cfg: CfgType,
#         logger: Optional[Logger] = None,
#         *args,
#         **kwargs,
#     ):
#         super().__init__(policy, cfg, logger)
#         self.policy = MLP([4, 100, 2], activation="relu")
#         self.q_net = self.policy
#         self.target_q_net = self.policy
#         self.loss_fn = torch.nn.functional.smooth_l1_loss

#     def act(self, state: torch.Tensor) -> torch.Tensor:
#         q = self.target_q_net(state)
#         action = torch.argmax(q)

#         # eposilon greedy
#         if torch.rand(1) < 0.1:
#             action = torch.randint(0, 2, (1,))
#         return action

#     def learn(self, batch: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
#         q = self.q_net(batch["state"])
#         q = q.gather(1, batch["action"].unsqueeze(0))
#         q_next = self.target_q_net(batch["next_state"])
#         q_next = q_next.max(1)[0]
#         q_target = batch["reward"] + 0.01 * q_next
#         loss = self.loss_fn(q, q_target)


In [4]:
agent = DQN(policy = [], optimizer=torch.optim.Adam, device="cuda", cfg={}, epsilon=0.1, gamma=0.01)

[WARNING] No policy is not defined in the config file.
[WARNING] No epsilon is not defined in the config file.
[WARNING] No gamma is not defined in the config file.
Sequential(
  (fc1): Linear(in_features=4, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=2, bias=True)
  (relu2): ReLU()
)
[INFO] Loading default lr. It is defined in the config file.


KeyError: 'lr is not defined in the config file.'

In [ ]:
memory = RandomMemory(capacity=10)

In [ ]:
state = env.reset()
state = state[0]

In [ ]:
for t in trange(1000):
    with torch.no_grad():

        action = agent.act(state[0])

    next_state, reward, terminated, truncated, info = env.step(action)
    memory.add(
        {
            "state": state,
            "action": action,
            "reward": reward,
            "next_state": next_state,
            "terminated": terminated,
            "truncated": truncated,
            "info": info,
        }
    )

    if not memory.is_full():
        continue
    else:
        batch = memory.sample(batch_size=3)
        agent.learn(batch)

    with torch.no_grad():
        if terminated.any() or truncated.any():
            states, infos = env.reset()

In [ ]:
a = torch.tensor([[0.0459, 0.0455],

        [0.0459, 0.0455],

        [0.0459, 0.0455]], device="cuda:0")

In [ ]:
b = torch.tensor([[0, 0, 0]], device='cuda:0')


In [ ]:
batch["action"]

In [ ]:
a.gather(1, b)